In [ ]:
import sys
sys.path.append('../../../../Ecallisto/analisis/pycallisto/src/') 
import pycallisto as pyc
import os
import requests
import pycallisto_utils as utils

In [ ]:
### PARA SRB ###
def get_txt_file(year, month, station):
    URL = 'http://soleil.i4ds.ch/solarradio/data/BurstLists/2010-yyyy_Monstein/'+year+'/e-CALLISTO_'+year+'_'+month+'.txt'
    # 2. download the data behind the URL
    response = requests.get(URL)

    # 3. Open the response into a new file
    txt_file=year+"_"+month+".txt"
    open(txt_file, "wb").write(response.content)
    
    f = open(txt_file, "r")
    lines=f.readlines()
    f.close()
    os.remove(txt_file)
    
    
    for line in lines:
        if line[0] != "#":
            row_info=line.split('\t')
            if len(row_info)>3:
                row_stations=row_info[3].split(', ')
                
                if len(row_stations)==1:
                    name=row_stations[0]
                    row_stations[0]=name[0:-1]
                
                try:
                    row_stations.index(station) 
                    day= row_info[0][6:8]
                    start_hour=row_info[1][0:2]
                    end_hour=row_info[1][6:8]
                    start_minute=row_info[1][3:5]
                    end_minute=row_info[1][9:]
                    
                    if start_hour==end_hour: # Just to avoid joining two separate fits files
                        if row_info[2]=='III':
                            get_fit_file(year, month, day, start_hour, start_minute, end_minute, station)
                except:
                    pass

In [ ]:
### PARA SRB ###
def get_fit_file(year, month, day, hour, start_minute, end_minute, station):
    
    date= year+month+day
    #time='141500'
    red_flag=0 #It shouldn't enter into while by default
    
    for i in range(0,60,15):
        limit_inf=str(i).zfill(2)
        limit_sup=str(i+15)

        if int(start_minute)>= int(limit_inf) and int(end_minute)<int(limit_sup):
            time=hour+limit_inf+'00'
            red_flag=1
    
    found=0
    while(red_flag==1):
        URL = 'http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/'+year+'/'+month+'/'+day+'/'+station+'_'+date+'_'+time+'_59.fit.gz'
        # 2. download the data behind the URL
        response = requests.get(URL)
        # 3. Open the response into a new file
        fit_file=station+'_'+date+'_'+time+"_01.fit"
        open(fit_file, "wb").write(response.content)
        #f.close()
        
        try:
            for file in os.listdir():
                if(file==fit_file):
                    #Gets fits file as an object
                    fit = pyc.PyCallisto.from_file(file)
            found=1
            red_flag=0
        except:
            new_second=time[4:]
            new_minute=time[2:4]
            new_hour=time[0:2]
            
            if int(new_second)==0: # Second and minute change
                new_second='59'
                if int(time[2:4])==0: # Minute and hour change
                    new_minute='59'
                    if int(time[0:2])==0:
                        red_flag=0 #00-> 23 implies day change, could also imply month and year change
                    else:
                        new_hour=str(int(time[0:2])-1).zfill(2)
                else:
                    new_minute=str(int(time[2:4])-1)
            elif int(time[4])==5:
                new_second= str(int(new_second)-1)
            
            elif int(new_second)==49 or (int(new_second)>0 and int(new_second)<=40 ):
                if int(new_second)==49:
                    new_second='01'
                    if int(new_minute)==59: # Minute and hour change
                        new_minute='00'
                        if int(new_hour)==23: 
                            red_flag=0
                        else:
                            new_hour=str(int(new_hour)+1).zfill(2)
                    else:
                        new_minute=str(int(new_minute)+1).zfill(2)         
                else:
                    new_second=str(int(new_second)+1).zfill(2)
                    
            if int(new_second)==41: # In case it doesn't find the fit file, to get out off the while loop
                red_flag=0
            os.remove(fit_file)
            time= new_hour+new_minute+new_second 
            
    if found==1:
        #os.remove(fit_file)
        open("FIT/"+fit_file, "wb").write(response.content)

In [ ]:
### PARA SRB ###
station='HUMAIN'
year='2023'

for j in range(1,12):
    month=str(j).zfill(2)
    get_txt_file(year, month, station)

In [ ]:
### PARA LOS QUE NO SON SRB ###
def get_fit_file_non_srb(year, month, day, station, counter):

    date=year+month+day
    
    for i in range(10):
        hour=str(i).zfill(2)
        
        for j in range(0,46,15):
            minute=str(j).zfill(2)
            time=hour+minute+'00'
            
            if counter>=26: # 50 when it's since july 2021. 33 for August 2022
                break
                
            URL ='http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/'+year+'/'+month+'/'+day+'/'+station+'_'+date+'_'+time+'_59.fit.gz'
            
            # 2. download the data behind the URL
            response = requests.get(URL)
            
            # 3. Open the response into a new file
            fit_file=station+'_'+date+'_'+time+"_59.fit"
            band=0
            
            #Checks if this fits file belongs to a SRB
            for file in os.listdir('FIT/'):
                    if (file==fit_file):
                        band+=1
                        
            #If it doesn't belong to a SRB, then do the following            
            if band==0:
                open("../NO ES SRB/FIT/"+fit_file, "wb").write(response.content)
                second = 0
                while second<11:
                    try:
                        for file in os.listdir('../NO ES SRB/FIT'):
                            if(file==fit_file):
                                fit = pyc.PyCallisto.from_file("../NO ES SRB/FIT/"+file)
                                counter+=1
                                second = 11

                    except:
                        os.remove("../NO ES SRB/FIT/"+fit_file)
                        second += 1
                        time=hour+minute+str(second).zfill(2)
                        URL ='http://soleil.i4ds.ch/solarradio/data/2002-20yy_Callisto/'+year+'/'+month+'/'+day+'/'+station+'_'+date+'_'+time+'_59.fit.gz'
                        # 2. download the data behind the URL
                        response = requests.get(URL)

                        # 3. Open the response into a new file
                        fit_file=station+'_'+date+'_'+time+"_59.fit"
                        if second < 11:
                            open("../NO ES SRB/FIT/"+fit_file, "wb").write(response.content)
                        

        if counter>=52:
            break
    return counter

In [ ]:
### PARA LOS QUE NO SON SRB ###
station='HUMAIN'
year='2023'
month='04'
counter=0

for j in range(1, 31):
    day = str(j).zfill(2)
    counter=get_fit_file_non_srb(year, month, day, station, counter)
    if counter>=26:
        break